In [2]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
import bitstring
from qiskit_aer import AerSimulator

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

from qiskit.circuit.library import UnitaryGate

In [3]:
matrix_h = ([[2**(-0.5),2**(-0.5)],[2**(-0.5),-2**(-0.5)]])
h_ideal = UnitaryGate(matrix_h)

matrix_cx = ([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
cx_ideal = UnitaryGate(matrix_cx)       #Erst Target, dann Control Qubit!!

matrix_x = ([[0,1],[1,0]])
x_ideal = UnitaryGate(matrix_x)

matrix_z = ([[1,0],[0,-1]])
z_ideal = UnitaryGate(matrix_z)

In [66]:
def code() -> QuantumCircuit:    #Returns the |0+> State
    qr = QuantumRegister(7, "q")
    cbit = ClassicalRegister(10,"c")
    qc = QuantumCircuit(qr,cbit)
    qc.h(0)
    qc.cx(0,1)
    qc.h(2)
    qc.cx(2,3)
    return qc

def X_L(qc: QuantumCircuit, q: list,n: int)-> QuantumCircuit:
    if n == 1:
        qc.x(q[0])
        qc.x(q[2])
    elif n == 2:
        qc.x(q[0])
        qc.x(q[1])

def Z_L(qc: QuantumCircuit, q: list, n: int)-> QuantumCircuit:
    if n == 1:
        qc.z(q[0])
        qc.z(q[1])
    elif n == 2:
        qc.z(q[0])
        qc.z(q[2])

def CNOT(control: int, q: list) -> QuantumCircuit:
    if control == 1:
        q[0], q[1] = q[1], q[0]
    elif control == 2:
        q[0], q[2] = q[2], q[0]

def H(qc: QuantumCircuit, q:list) -> QuantumCircuit:
    for i in range(4):
        qc.h(q[i])
    q[1], q[2] = q[2], q[1]

def CZ_L(qc: QuantumCircuit, q: list)-> QuantumCircuit:
    for i in range(4):
        qc.s(q[i])
    qc.z(q[1])
    qc.z(q[2])

def S_L(qc: QuantumCircuit, q: list, pos: int)-> QuantumCircuit:
    qc.reset(q[5])
    qc.reset(q[4])

    qc.h(q[5])
    qc.s(q[5])

    qc.cx(q[5],q[4])

    if pos == 1:
        qc.cx(q[0],q[4])
        qc.cx(q[1],q[4])
    elif pos == 2:
        qc.cx(q[0],q[4])
        qc.cx(q[2],q[4])

    qc.measure(q[4],0)

    if pos == 1:
        with qc.if_test((0,1)):
            qc.z(q[0])
            qc.z(q[1])
    elif pos == 2:
        with qc.if_test((0,1)):
            qc.z(q[0])
            qc.z(q[2])


    qc.h(q[5])
    qc.measure(q[5],0)

    if pos == 1:
        with qc.if_test((0,1)):
            qc.z(q[0])
            qc.z(q[1])
    elif pos == 2:
        with qc.if_test((0,1)):
            qc.z(q[0])
            qc.z(q[2])

def T_L(qc: QuantumCircuit, q: list, pos: int)-> QuantumCircuit:
    qc.reset(q[6])
    qc.reset(q[5])
    qc.reset(q[4])

    qc.h(q[6])
    qc.t(q[6])

    qc.cx(q[6],q[4])

    if pos == 1:
        qc.cx(q[0],q[4])
        qc.cx(q[1],q[4])
    elif pos == 2:
        qc.cx(q[0],q[4])
        qc.cx(q[2],q[4])

    qc.measure(q[4],0)

    if pos == 1:
        with qc.if_test((0,1)):
            qc.reset(q[4])
            qc.h(q[5])
            qc.s(q[5])
            qc.cx(q[0],q[4])
            qc.cx(q[1],q[4])
            qc.cx(q[5],q[4])
            qc.measure(q[4],0)
            with qc.if_test((0,1)):
                qc.z(q[0])
                qc.z(q[1])
            qc.h(q[5])
            qc.measure(q[5],0)
            with qc.if_test((0,1)):
                qc.z(q[0])
                qc.z(q[1])

    elif pos == 2:
        with qc.if_test((0,1)):
            qc.reset(q[4])
            qc.h(q[5])
            qc.s(q[5])
            qc.cx(q[0],q[4])
            qc.cx(q[2],q[4])
            qc.cx(q[5],q[4])
            qc.measure(q[4],0)
            with qc.if_test((0,1)):
                qc.z(q[0])
                qc.z(q[2])
            qc.h(q[5])
            qc.measure(q[5],0)
            with qc.if_test((0,1)):
                qc.z(q[0])
                qc.z(q[2])

    qc.h(q[6])
    qc.measure(q[6],0)

    if pos == 1:
        with qc.if_test((0,1)):
            qc.z(q[0])
            qc.z(q[1])
    elif pos == 2:
        with qc.if_test((0,1)):
            qc.z(q[0])
            qc.z(q[2])

def adj_T_L(qc: QuantumCircuit, q: list, pos: int)-> QuantumCircuit:
    qc.reset(q[6])
    qc.reset(q[5])
    qc.reset(q[4])

    qc.h(q[6])
    qc.tdg(q[6])

    qc.cx(q[6],q[4])

    if pos == 1:
        qc.cx(q[0],q[4])
        qc.cx(q[1],q[4])
    elif pos == 2:
        qc.cx(q[0],q[4])
        qc.cx(q[2],q[4])

    qc.measure(q[4],0)

    if pos == 1:
        with qc.if_test((0,1)):
            qc.reset(q[4])
            qc.h(q[5])
            qc.sdg(q[5])
            qc.cx(q[0],q[4])
            qc.cx(q[1],q[4])
            qc.cx(q[5],q[4])
            qc.measure(q[4],0)
            with qc.if_test((0,1)):
                qc.z(q[0])
                qc.z(q[1])
            qc.h(q[5])
            qc.measure(q[5],0)
            with qc.if_test((0,1)):
                qc.z(q[0])
                qc.z(q[1])

    elif pos == 2:
        with qc.if_test((0,1)):
            qc.reset(q[4])
            qc.h(q[5])
            qc.sdg(q[5])
            qc.cx(q[0],q[4])
            qc.cx(q[2],q[4])
            qc.cx(q[5],q[4])
            qc.measure(q[4],0)
            with qc.if_test((0,1)):
                qc.z(q[0])
                qc.z(q[2])
            qc.h(q[5])
            qc.measure(q[5],0)
            with qc.if_test((0,1)):
                qc.z(q[0])
                qc.z(q[2])

    qc.h(q[6])
    qc.measure(q[6],0)

    if pos == 1:
        with qc.if_test((0,1)):
            qc.z(q[0])
            qc.z(q[1])
    elif pos == 2:
        with qc.if_test((0,1)):
            qc.z(q[0])
            qc.z(q[2])

def CS_L(qc: QuantumCircuit, q: list, control: int, target: int)-> QuantumCircuit:
    T_L(qc, q, control)
    T_L(qc, q, target)
    CNOT(control, q)
    adj_T_L(qc, q, target)
    CNOT(control, q)

def readout(qc: QuantumCircuit, q: list, pos: int,shots: int):
    qc.reset(q[4])
    if pos == 1:
        qc.cx(q[0],q[4])
        qc.cx(q[1],q[4])
    elif pos == 2:
        qc.cx(q[0],q[4])
        qc.cx(q[2],q[4])
    qc.measure(q[4],0)
    sim = AerSimulator()
    job = sim.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    return counts

In [69]:
q = [0,1,2,3,4,5,6]         # = q0, q1, q2, q3, a, s_m, t_m    in this order

qc = code()

X_L(qc, q, 1)

CS_L(qc, q, 1, 2)
CS_L(qc, q, 1, 2)

H(qc, q)

counts = readout(qc, q, 2, 1000)

print(counts)

{'0000000001': 1000}


In [ ]:
#iQPE first iteration

qc = code()

